In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Environment
---

In [ ]:
%run env.py
%run rl-helpers.py
from IPython.display import clear_output
import time

# Create arena
env = DeliveryDrones(['A', 'B', 'C'])
rewards = None
while True:
    clear_output(wait=True)
    env.render()
    actions = {
        'A': env.sample(),
        'B': env.sample(),
        'C': env.sample()
    }
    print(rewards)
    print({n: Action(i) for n, i in actions.items()})
    time.sleep(0.1)
    
    next_states, rewards, done, _ = env.step(actions)

Graphics
---

In [ ]:
# TODO

# Drone, pickup, dropoff, collision ~ simple geometric shapes
# https://image.freepik.com/free-vector/simple-geometric-shapes-background_1168-371.jpg
# Animation: slightly growing/shrinking to simulate up/down movement with shade

# Desired output
# https://img.deszone.net/2018/05/simple-geometric-shapes-free-vector-pattern4.jpg
# https://as1.ftcdn.net/jpg/01/72/82/18/500_F_172821814_Oyl3cNYBcigDpeCzehbAQghLxJILrZA5.jpg

# Other ideas
# Drones leave a fading trace